## Packages

In [31]:
import selenium
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re

import numpy as np
import pandas as pd

## Creating Location DataFrame

In [326]:
location = pd.read_csv("location_url.csv")
location = pd.DataFrame(location)

## Intiating Driver

In [4]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
# driver.get("https://www.google.com/map")
driver.get(location.URL[1])
driver.maximize_window()
sleep(4)

## Scrolling

In [5]:
# scroll = driver.find_element_by_xpath("/html/body/jsl/div[3]/div[9]/div[8]/div/div[1]")
# scroll.send_keys(Keys.HOME) 
# scroll.send_keys(Keys.END)
# scroll.send_keys(Keys.UP) 
# scroll.send_keys(Keys.DOWN)
# scroll.send_keys(Keys.PAGE_UP)
# scroll.send_keys(Keys.PAGE_DOWN)

## More Reviews Buttom

In [7]:
# Scoll in Pane
scroll = driver.find_element_by_xpath("/html/body/jsl/div[3]/div[9]/div[8]/div/div[1]")

In [8]:
# More Reviews Button
more_reviews_button = driver.find_element_by_xpath("/html/body/jsl/div[3]/div[9]/div[8]/div/div[1]/div/div/div[51]/div/div/button")

In [9]:
# finds more reviews button
while more_reviews_button.is_displayed() == False:
    scroll.send_keys(Keys.PAGE_DOWN)

# clicks more reviews button
more_reviews_button.click()  

In [263]:
more_text_button = driver.find_elements_by_xpath("/html/body/jsl/div[3]/div[9]/div[8]/div/div[1]/div/div/div[3]/div[9]/div[*]/div/div[3]/div[3]/jsl/button")

for button in more_text_button:
    button.click()  

# Data Frame Reviews

In [285]:
reviews = pd.DataFrame()

## Review Scraper

In [287]:
# title = driver.find_element_by_class_name("section-review-title")
# title.text
temp = []
names = driver.find_elements_by_class_name("section-review-title")
for name in names:
    temp.append(name.text)

reviews["name"] = temp

In [288]:
# text = driver.find_element_by_class_name("section-review-text")
# text.text

temp = []

texts = driver.find_elements_by_class_name("section-review-text")
for text in texts:
    temp.append(text.text)

reviews["text"] = temp
    

In [289]:
reviews["date_scraped"] = "03/26/2021"

temp = []

dates = driver.find_elements_by_class_name("section-review-publish-date")
for date in dates:
    temp.append(date.text)

reviews["date_published"] = temp

In [290]:

temp = []

ratings = driver.find_elements_by_class_name("section-review-stars")
for rating in ratings:
    temp.append(int(rating.get_attribute("aria-label")[1]))

reviews["rating"] = temp

In [291]:
temp = []
reviews["likes"] = 0
likes = driver.find_elements_by_class_name("section-review-interaction-button")
for like in likes:
    temp.append(like.text)
temp = temp[::2]
temp = [0 if i == "Like" else int(i) for i in temp]
for i, x in enumerate(temp):
    reviews.likes[i] = x


<ipython-input-291-2cc15be73208>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews.likes[i] = x


In [298]:
temp =[]

contents = driver.find_elements_by_class_name("section-review-content")
for content in contents:
    temp.append(content.text)
temp1 = pd.DataFrame(temp, columns=["full"])


temp1["split"] = temp1["full"].str.split("\n")
temp1["subtitle"] = [i[1] if len(i) >=3 else "" for i in temp1.split]
temp1["local_guide"] = [1 if "Local" in i else 0 for i in temp1.subtitle]
temp1["review_count"] = [int(re.findall(r'\d+',i)[0]) if len(re.findall(r'\d+',i))>0 else 0 for i in temp1.subtitle]

review_old = reviews.copy()

reviews = reviews.merge(temp1[["local_guide","review_count"]], left_index=True,right_index=True)


In [328]:
temp = []
profiles = driver.find_elements_by_xpath("/html/body/jsl/div[3]/div[9]/div[8]/div/div[1]/div/div/div[3]/div[9]/div[*]/div/div[3]/div[2]/div/div/a")
for profile in profiles:
    temp.append(profile.get_attribute("href"))

reviews["profile_url"] = temp


In [355]:
reviews["location"] = location["Full Address"][0]
reviews["location_url"] = location["URL"][0]

In [356]:
reviews

,name,text,date_scraped,date_published,rating,likes,local_guide,review_count,profile_url,location,location_url
0,Job Req,Very badly managed club... Had a very bad rece...,03/26/2021,3 weeks ago,1,0,0,11,https://www.google.com/maps/contrib/1055815481...,BJ's Wholesale 278 Middlesex Avenue Medford M...,https://www.google.com/maps/place/BJ's+Wholesa...
1,Barbara Perez,I always find something unexpected and unique ...,03/26/2021,4 months ago,4,0,1,93,https://www.google.com/maps/contrib/1003703133...,BJ's Wholesale 278 Middlesex Avenue Medford M...,https://www.google.com/maps/place/BJ's+Wholesa...
2,Jenny Jay Rapirap,Did not accept a return (unopened set of tooth...,03/26/2021,3 months ago,1,0,1,172,https://www.google.com/maps/contrib/1136537256...,BJ's Wholesale 278 Middlesex Avenue Medford M...,https://www.google.com/maps/place/BJ's+Wholesa...
3,V P,"Woman not wearing mask, reported it to manager...",03/26/2021,a month ago,1,0,0,5,https://www.google.com/maps/contrib/1056014347...,BJ's Wholesale 278 Middlesex Avenue Medford M...,https://www.google.com/maps/place/BJ's+Wholesa...
4,Butch Hyde,Great Sale,03/26/2021,3 years ago,1,0,0,9,https://www.google.com/maps/contrib/1061470280...,BJ's Wholesale 278 Middlesex Avenue Medford M...,https://www.google.com/maps/place/BJ's+Wholesa...
...,...,...,...,...,...,...,...,...,...,...,...
196,William H. Creitz,,03/26/2021,3 years ago,5,0,1,78,https://www.google.com/maps/contrib/1157347278...,BJ's Wholesale 278 Middlesex Avenue Medford M...,https://www.google.com/maps/place/BJ's+Wholesa...
197,Sonia Ortiz,,03/26/2021,3 years ago,5,0,1,1,https://www.google.com/maps/contrib/1123192890...,BJ's Wholesale 278 Middlesex Avenue Medford M...,https://www.google.com/maps/place/BJ's+Wholesa...
198,William Gokey,,03/26/2021,3 years ago,5,0,0,6,https://www.google.com/maps/contrib/1089534147...,BJ's Wholesale 278 Middlesex Avenue Medford M...,https://www.google.com/maps/place/BJ's+Wholesa...
199,Marlon Williams,,03/26/2021,3 years ago,5,0,0,0,https://www.google.com/maps/contrib/1116221582...,BJ's Wholesale 278 Middlesex Avenue Medford M...,https://www.google.com/maps/place/BJ's+Wholesa...


In [362]:
reviews.to_csv(f'{location["Company"][0]}_{"000"}_Reviews.csv', index=False)